# Enhanced Derivation and Simulation of Boltzmann-Arrhenius and Eyring Equations
## for the Temperature Dependence of Enzyme Kinetics

This notebook illustrates how the **Arrhenius** and **Eyring** equations describe the temperature dependence of enzyme kinetics, while also taking into account thermal denaturation at higher temperatures. We further illustrate these concepts by referencing a recent review figure (shown below) that provides additional visual context for both **empirical** (Arrhenius) and **transition-state** (Eyring) perspectives.

## Attached Graphic (from Recent Review)

Below is the figure you shared (Figure 1), which shows:
- **(a)** The Arrhenius law from equilibrium thermodynamics, 
- **(b)** Enzyme kinetics and the transition state, 
- **(c)** Eyring equation and measurement of thermodynamic quantities by temperature variation,
- **(d)** Temperature dependence of Michaelis–Menten kinetics,
- **(e)** Enzymatic rate enhancement via enthalpic or entropic pathways.

It nicely complements the code and discussion in this notebook:

![Figure from Review](<PUT_A_LINK_OR_LOCAL_PATH_HERE>)  

*In panel (a), you see the classical Arrhenius plot (\(\ln(k)\) vs \(1/T\)). In panel (b), the enzyme’s rate enhancement is framed via transition-state theory. Panel (c) introduces the Eyring equation. Panel (d) explores how temperature influences Michaelis–Menten parameters. Panel (e) compares enthalpic vs. entropic contributions to enzymatic rate enhancement.*

Below, we provide both **theoretical derivation** and **Python-based simulation** of these models, building on the same fundamentals illustrated by this figure.

### 1. Introduction and Historical Context

#### 1.1. Arrhenius Equation

One of the earliest attempts to capture the temperature dependence of reaction rates was pioneered by **Jacobus Henricus van ’t Hoff** (1852–1911), who observed an exponential increase in the rate of chemical reactions with increasing temperature. Later, **Svante Arrhenius** (1859–1927) formalized this relationship in the well-known **Arrhenius Equation**:

$$\displaystyle k = A \exp\Bigl( -\frac{E_a}{RT} \Bigr),$$

where  
- \(k\) is the rate constant,  
- \(A\) is the pre-exponential (frequency) factor,  
- \(E_a\) is the activation energy,  
- \(R\) is the universal gas constant, and  
- \(T\) is the absolute temperature (in Kelvin).

For **enzyme-catalyzed reactions**, the same principle applies for the **catalytic rate constant** \(k_{\mathrm{cat}}\), but enzymes also have a **thermal denaturation** process at higher temperatures, causing activity to drop above a certain threshold.

#### 1.2. Eyring Equation (Transition State Theory)

In the 1930s, **Henry Eyring** (1901–1981) and others developed the **Transition State Theory (TST)**. The key result of TST is the **Eyring-Polanyi Equation**, usually written in the form:

$$\displaystyle k = \kappa \frac{k_B \, T}{h} \exp\Bigl( \frac{\Delta S^\ddagger}{R}\Bigr) \exp\Bigl( - \frac{\Delta H^\ddagger}{RT} \Bigr),$$

where  
- \(\kappa\) is the transmission coefficient (often assumed to be ~1 for many simple reactions),  
- \(k_B\) is the Boltzmann constant,  
- \(h\) is Planck’s constant,  
- \(\Delta S^\ddagger\) is the entropy of activation,  
- \(\Delta H^\ddagger\) is the enthalpy of activation, and  
- \(R\) and \(T\) are as before.

This equation is sometimes called the “**Eyring equation**” and offers a thermodynamics-based perspective. It reduces to an Arrhenius-like form when you group constants appropriately.

#### 1.3. Application to Enzyme Kinetics

For an **enzyme-catalyzed reaction**, the rate can be written as:

$$\displaystyle \text{Rate} = k_\text{cat}(T) \times [\text{E}] \times f([\text{S}]),$$

where:
- \(k_\text{cat}(T)\) is the turnover number, strongly dependent on \(T\),  
- \([\text{E}]\) is the enzyme concentration,  
- \(f([\text{S}])\) is a function of substrate concentration (such as the Michaelis-Menten form \(\tfrac{[\text{S}]}{K_M + [\text{S}] }\)).

We can model \(k_\text{cat}(T)\) using either the **Arrhenius Equation** or the **Eyring Equation**. However, because real enzymes **denature** at higher temperatures, we often observe a **bell-shaped** temperature activity profile.

In this notebook, we will:
1. Demonstrate both the **Arrhenius** and **Eyring** forms for the temperature dependence of \(k_\text{cat}\).  
2. Include a simple logistic denaturation function to mimic the loss of enzyme activity at high temperature.  
3. Visualize the resulting enzyme activity curves and explore parameter effects.

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# For better plotting aesthetics
# plt.style.use('seaborn-darkgrid')

## 2. Constants and Helper Functions

We'll define relevant physical constants and implement two functions for the temperature-dependent rate constants:

1. **Arrhenius Rate**:  
$$\displaystyle k_{\text{Arrhenius}}(T) = A \exp\Bigl(-\frac{E_a}{RT}\Bigr).$$

2. **Eyring Rate**:  
$$\displaystyle k_{\text{Eyring}}(T) = \kappa \frac{k_B T}{h} \exp\Bigl(\frac{\Delta S^\ddagger}{R}\Bigr) \exp\Bigl(-\frac{\Delta H^\ddagger}{RT}\Bigr).$$

(Assume \(\kappa \approx 1\) for simplicity.)

In both cases, we’ll multiply by a logistic **denaturation factor** to represent the fraction of enzyme that remains active at high temperature.  

Refer to **panel (c)** in the attached figure for how the Eyring equation can be fitted to data to extract thermodynamic parameters.

In [3]:
# Physical constants
R = 8.314            # J/(mol*K) - Universal gas constant
k_B = 1.380649e-23   # J/K - Boltzmann constant
h = 6.62607015e-34   # J*s - Planck's constant

def arrhenius_rate(T_celsius, A=1e7, Ea=5e4, T_denature=60.0, slope=0.5):
    """
    Returns the enzyme-catalyzed reaction rate (arbitrary units)
    using the Arrhenius form combined with a logistic denaturation factor.
    
    Parameters
    ----------
    T_celsius : float
        Temperature in Celsius
    A : float
        Pre-exponential factor for Arrhenius
    Ea : float
        Activation energy in J/mol
    T_denature : float
        Approximate denaturation temperature in Celsius
    slope : float
        Controls the steepness of the logistic denaturation
    
    Returns
    -------
    rate : float
        Modeled enzyme rate at the given temperature
    """
    T_kelvin = T_celsius + 273.15
    
    # Arrhenius part
    k_cat = A * np.exp(-Ea / (R * T_kelvin))
    
    # Logistic denaturation factor
    fraction_active = 1.0 / (1.0 + np.exp(slope * (T_celsius - T_denature)))
    
    return k_cat * fraction_active

def eyring_rate(T_celsius, dH=5e4, dS=-100, T_denature=60.0, slope=0.5):
    """
    Returns the enzyme-catalyzed reaction rate (arbitrary units)
    using the Eyring (transition state) form combined with a logistic denaturation factor.
    
    Parameters
    ----------
    T_celsius : float
        Temperature in Celsius
    dH : float
        Enthalpy of activation (J/mol)
    dS : float
        Entropy of activation (J/(mol*K))
    T_denature : float
        Approximate denaturation temperature in Celsius
    slope : float
        Controls the steepness of the logistic denaturation
    
    Returns
    -------
    rate : float
        Modeled enzyme rate at the given temperature
    """
    T_kelvin = T_celsius + 273.15
    
    # Eyring part (assuming transmission coefficient ~ 1)
    # kB*T/h in s^-1, multiplied by exponent factors
    k_cat = (k_B * T_kelvin / h) * np.exp(dS / R) * np.exp(-dH / (R * T_kelvin))
    
    # Logistic denaturation factor
    fraction_active = 1.0 / (1.0 + np.exp(slope * (T_celsius - T_denature)))
    
    return k_cat * fraction_active


## 3. Interactive Plots

Below, we create two interactive plots:
1. **Arrhenius-based** temperature dependence.  
2. **Eyring-based** temperature dependence.

In both cases, you can adjust parameters such as the activation energy/enthalpy (\(E_a\) or \(\Delta H^\ddagger\)), the entropy of activation (\(\Delta S^\ddagger\)), and the denaturation temperature or slope to see how the rate curve is affected.

Refer to **panel (d)** in the figure for how Michaelis–Menten kinetics specifically depends on \(k_{\mathrm{cat}}\) and \(K_M\) with temperature, but here we focus on the temperature dependence of \(k_{\mathrm{cat}}\) itself for simplicity.

In [4]:
def plot_arrhenius(A=1e7, Ea=5e4, T_denature=60.0, slope=0.5):
    T = np.linspace(0, 100, 200)  # Temperature in Celsius
    rates = [arrhenius_rate(t, A=A, Ea=Ea, T_denature=T_denature, slope=slope) for t in T]
    
    plt.figure(figsize=(8,5))
    plt.plot(T, rates, label='Arrhenius-based Enzyme Rate')
    plt.axvline(T_denature, color='r', linestyle='--', label='Denaturation Temp')
    plt.xlabel('Temperature (°C)')
    plt.ylabel('Rate (arbitrary units)')
    plt.title('Arrhenius + Denaturation Model')
    plt.legend()
    plt.show()

def plot_eyring(dH=5e4, dS=-100, T_denature=60.0, slope=0.5):
    T = np.linspace(0, 100, 200)  # Temperature in Celsius
    rates = [eyring_rate(t, dH=dH, dS=dS, T_denature=T_denature, slope=slope) for t in T]
    
    plt.figure(figsize=(8,5))
    plt.plot(T, rates, label='Eyring-based Enzyme Rate')
    plt.axvline(T_denature, color='r', linestyle='--', label='Denaturation Temp')
    plt.xlabel('Temperature (°C)')
    plt.ylabel('Rate (arbitrary units)')
    plt.title('Eyring + Denaturation Model')
    plt.legend()
    plt.show()

interact(plot_arrhenius,
         A=FloatSlider(value=1e7, min=1e5, max=1e9, step=1e5, continuous_update=False),
         Ea=FloatSlider(value=5e4, min=1e4, max=2e5, step=1e4, continuous_update=False),
         T_denature=FloatSlider(value=60, min=20, max=100, step=1.0, continuous_update=False),
         slope=FloatSlider(value=0.5, min=0.1, max=2.0, step=0.1, continuous_update=False));

interact(plot_eyring,
         dH=FloatSlider(value=5e4, min=1e4, max=2e5, step=1e4, continuous_update=False),
         dS=FloatSlider(value=-100, min=-300, max=300, step=10, continuous_update=False),
         T_denature=FloatSlider(value=60, min=20, max=100, step=1.0, continuous_update=False),
         slope=FloatSlider(value=0.5, min=0.1, max=2.0, step=0.1, continuous_update=False));

interactive(children=(FloatSlider(value=10000000.0, continuous_update=False, description='A', max=1000000000.0…

interactive(children=(FloatSlider(value=50000.0, continuous_update=False, description='dH', max=200000.0, min=…

## 4. Discussion

1. **Arrhenius vs. Eyring**  
   - The **Arrhenius Equation** is an empirical formula that captures the exponential increase of reaction rate with temperature.  
   - The **Eyring Equation** derives from **Transition State Theory**, relating rate constants to thermodynamic parameters of the transition state (\(\Delta H^\ddagger\), \(\Delta S^\ddagger\)), as depicted in panels (b) and (c) of the figure.

2. **Role of Denaturation**  
   - Both models in this notebook include a *logistic* denaturation factor to reduce active enzyme at high temperatures. Real systems can have more intricate denaturation kinetics, but this simple approach captures the key idea: above a certain threshold, enzyme function declines rapidly.

3. **Bell-Shaped Curve**  
   - Combining an **Arrhenius- or Eyring-like increase** at moderate temperatures with **denaturation** at higher temperatures typically yields the **bell-shaped** temperature-activity curve depicted in many enzyme-kinetics references.  
   - As shown in panel (d), enzymatic rates may also be viewed in the context of substrate concentration and Michaelis–Menten parameters, but the main driver at high temperature is denaturation.

4. **Parameter Interpretation**  
   - **Arrhenius**: \(E_a\) (J/mol) is the activation energy barrier.  
   - **Eyring**: \(\Delta H^\ddagger\) (J/mol) is the enthalpy of activation; \(\Delta S^\ddagger\) (J/(mol·K)) is the entropy of activation. A large positive \(\Delta S^\ddagger\) can lead to higher rates, while a large positive \(\Delta H^\ddagger\) lowers the rate.

5. **Historical Significance**  
   - **Arrhenius** (1889) built on **van ’t Hoff’s** work on temperature dependence of equilibrium constants and reaction rates.  
   - **Henry Eyring** (1935) developed Transition State Theory, leading to the **Eyring-Polanyi equation**, providing a more fundamental theoretical underpinning for temperature-dependent rate processes in chemistry and biochemistry.

## 5. Conclusion

In this notebook, we have:
- Incorporated insights from a **recent review figure** (panels (a)–(e)) showing Arrhenius, Eyring, and Michaelis–Menten perspectives.  
- Shown how **Arrhenius** and **Eyring** equations each describe temperature-dependent reaction rates.  
- Applied both to a simplified **enzyme kinetics** scenario, including a denaturation term.  
- Provided interactive sliders to explore how parameters like \(E_a\), \(\Delta H^\ddagger\), \(\Delta S^\ddagger\), and the denaturation threshold affect the enzyme’s activity profile.

The combination of these models, especially with the logistic denaturation factor, nicely illustrates the **bell-shaped** temperature dependence typically observed in real enzymes. As the attached figure shows, such analyses can be extended to full **Michaelis–Menten** kinetics (panel d) and to distinguishing **enthalpic** vs. **entropic** components of catalysis (panel e).

### References

1. Arrhenius, S. (1889). *Über die Reaktionsgeschwindigkeit bei Inversion von Rohrzucker durch Säuren*. **Z. Phys. Chem.** 4, 226–248.  
2. van ’t Hoff, J. H. (1884). *Études de dynamique chimique*. Frederik Muller & Cie.  
3. Eyring, H. (1935). The Activated Complex in Chemical Reactions. **J. Chem. Phys.** 3(2), 107–115.  
4. Polanyi, M. (1935). **Z. Phys. Chem. B** 28, 309–318.  
5. Johnson, K. A., & Goody, R. S. (2011). The Original Michaelis Constant: Translation of the 1913 Michaelis–Menten Paper. **Biochemistry**, 50(39), 8264–8269.  

> **Note:** For full interactivity (slider-based plots), run this notebook in an environment that supports `ipywidgets` such as **Jupyter Notebook** or **JupyterLab**.